In [8]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
import io
import os
from global_variables import *

In [9]:
# 02_match_poi_sod.ipynb creates this file
poi_sod_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/poi_sod_4.csv.gz" 
poi_sod = pd.read_csv(poi_sod_file, compression='gzip')

In [10]:
date_start = '2019-01-01'
date_end = '2020-03-01'

file_suffix = date_start.replace('-', '')+"_"+date_end.replace('-', '')
file_suffix

'20180101_20181201'

In [7]:
# results = requests.get(url=MONTHLY_FT_PATTERNS_PRODUCT_API_PATH,
#                        params={'partition_key_after': date_start,   # optionally set date value here
#                                'partition_key_before': date_end}, # optionally set date value here
#                        headers={'X-API-KEY': API_KEY,
#                                 'accept': 'application/json'
#                                })
# response_json = results.json()

In [11]:
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/filtered_ft_data_'+file_suffix+'.csv'
page = 1
download_count = 0
while True:
    print(page)
    results = requests.get(url=MONTHLY_FT_PATTERNS_PRODUCT_API_PATH,
                       params={'page': page,
                               'partition_key_after': date_start,   # optionally set date value here
                               'partition_key_before': date_end}, # optionally set date value here
                       headers={'X-API-KEY': API_KEY,
                                'accept': 'application/json'
                               })
    response_json = results.json()

    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        filtered_df = df[df['PLACEKEY'].isin(poi_sod['PLACEKEY'])]
    
        if os.path.isfile(ft_data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(ft_data_path, mode='a', header=header_option, index=False)
        
        download_count += 1
        
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1   

1


In [5]:
ft_data = pd.read_csv(ft_data_path,low_memory=False)

In [6]:
ft_data = ft_data.dropna(subset=['VISITOR_HOME_CBGS'])
ft_data = ft_data.reset_index()

In [7]:
with gzip.open(ft_data_path+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    ft_data.to_csv(gzipped_file, index=False)